In [ ]:
import os
import PySpin
import cv2
import time
import numpy as np

# Manager class
class PySpinManager(object):
    def __init__(self):
        # Retrieve singleton reference to system object
        self.system = PySpin.System.GetInstance()
        # Get current library version
        version = self.system.GetLibraryVersion()
        print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))
        
        # Get camera list
        cam_list = self.system.GetCameras()
        num_cameras = cam_list.GetSize()
        print('Number of cameras detected: %d' % num_cameras)
        self.cams = []
        self.serial2idx = {}
        for i in range(num_cameras):
            self.cams.append(cam_list.GetByIndex(i))
            self.cams[-1].Init()
            serial = self.cams[-1].DeviceSerialNumber.ToString()
            self.serial2idx[serial] = i
            self.cams[-1].DeInit()

        cam_list.Clear()
        
    
    def get_camera(self, idx):
        if isinstance(idx, str):
            idx = self.serial2idx[idx]
        elif isinstance(idx, int):
            pass
        else:
            raise PySpin.SpinnakerException('idx should be serial(str) or index(int)')
        
        cam = self.cams[idx]
        self.cams[idx] = None
        return PySpinCam(cam)
    
    def get_multi_camera(self, idx):
        if not isinstance(idx, list):
            raise PySpin.SpinnakerException('idx should be list of index or serial')
        multi_cap = PySpinMultiCam()
        for it in idx:
            multi_cap.add_camera(self.get_camera(it))
        return multi_cap
        
        
    def release(self):
        # Release cameras
        for i in range(len(self.cams)):
            self.cams[i] = None

        # Release system instance
        self.system.ReleaseInstance()
        self.system = None
        print('release everything')
        
    def __del__(self):
        self.release()

        
# Single camera class
class PySpinCam(object):
    def __init__(self, cam):
        if cam is None:
            print('cam is None. Already used in other object')
            return
        self.cam = cam
        self.cam.Init()
        self.cam.PixelFormat.SetValue(PySpin.PixelFormat_BayerGB8)
        print('Pixel format:', self.cam.PixelFormat.GetCurrentEntry().GetSymbolic())
        device_model = self.cam.DeviceModelName.ToString()
        print('Device model:', device_model)
        # Get serial number
        print('Serial number:', self.cam.DeviceSerialNumber.ToString())
        
        # image acquisition
        self.cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
        # trigger setting
        self._set_software_trigger()
        # white balance setting
        self._set_white_balance_ratio(1.06)
        
        #  start capturing
        self.cam.BeginAcquisition()
        self.isGrab = False

    def read(self):
        self.grab()
        ret, img = self.retrieve()
        return ret, img
    
    def grab(self):
        self.cam.TriggerSoftware()
        self.isGrab = True
        
    def retrieve(self):
        if not self.isGrab:
            print('Call grab first')
            return False, np.array([])
        
        image_result = self.cam.GetNextImage()
        if image_result.IsIncomplete():
            return False, np.array([])
        
        # Bayer to color
        img = image_result.GetNDArray()
#     image_converted = image_result.Convert(PySpin.PixelFormat_BGR8, PySpin.HQ_LINEAR) # slow
#     img = image_converted.GetNDArray()
        img = cv2.cvtColor(img, cv2.COLOR_BAYER_GR2BGR) # bayer pattern is correct?
        image_result.Release()
        isGrab = False
        return True, img

    
    def release(self):
        self.cam.EndAcquisition()
        self.cam.DeInit()
        self.cam = None
        
    def __del__(self):
        self.release()
        
    ##########################
    ## setting
    def _set_software_trigger(self):
        self.cam.TriggerMode.SetValue(PySpin.TriggerMode_Off)
        self.cam.TriggerSource.SetValue(PySpin.TriggerSource_Software)
        self.cam.TriggerMode.SetValue(PySpin.TriggerMode_On)
    
    def _set_white_balance_ratio(self, val):
        self.cam.BalanceRatioSelector.SetValue(PySpin.BalanceRatioSelector_Red)
        self.cam.BalanceRatio.SetValue(val)
        
        
        
# Multiple camera class (software sync)
class PySpinMultiCam(object):
    def __init__(self):
        self.spincams = []
        
    def add_camera(self, spincam):
        self.spincams.append(spincam)
        
    def read(self):
        self.grab()
        ret, imgs = self.retrieve()
        return ret, imgs
    
    def grab(self):
        for it in self.spincams:
            it.grab()

    def retrieve(self):
        ret = True
        imgs = []
        for it in self.spincams:
            flag, img = it.retrieve()
            ret &= flag
            imgs.append(img)
        return ret, imgs

    def release(self):
        for it in self.spincams:
            it.release()
            
    def __del__(self):
        self.release()

In [ ]:
manager = PySpinManager()

In [ ]:
STEREO = False

if STEREO:
    cap = manager.get_multi_camera([0, 1])
else:
    cap = manager.get_camera(0)
    
print('#####################')
print('### start capturing')
elps = []
save_imgs = []
while True:
    start = time.time()
    
    if STEREO:
        ret, imgs = cap.read()
    else:
        ret, img = cap.read()
        imgs = [img]
    
    for i, img in enumerate(imgs):
        img = cv2.resize(img, (640, 640))
        cv2.imshow('img{}'.format(i), img)
    key= cv2.waitKey(20)
    elps.append((time.time()-start))
    
    if key==ord('s'):
        cv2.waitKey()
        save_imgs.append(imgs)
        
    if key==27:
        break

cv2.destroyAllWindows()
print('### finish capturing')
print('#####################')
fps = []
for it in elps:
    fps.append(1/it)
print('Average fps:', np.array(fps).mean())

cap.release()